In [2]:
import pandas as pd
import os
from natsort import natsorted
import glob
import re
from pathlib import Path
import subprocess
import gzip

In [2]:
def create_empty_fastq_gz(filename):
    with gzip.open(filename, 'wb') as f:
        pass  # Writing nothing creates an empty file

In [12]:
#Get rid of undemultiplexed files
r1_files = natsorted(glob.glob("20250417_demux/*R1_001_*.fastq.gz"))
r1_files = [ x for x in r1_files if "UNKNOWN" not in x ]
r2_files = natsorted(glob.glob("20250417_demux/*R2_001_*.fastq.gz"))
r2_files = [ x for x in r2_files if "UNKNOWN" not in x ]

print(len(r1_files), len(r2_files))

2304 2304


In [ ]:
#filter out unpaired reads
def run_seqkit_pair(r1, r2):             
    command = f"seqkit pair -1 {r1} -2 {r2} -O OUTPUT_DIR"
    return command   

In [ ]:
with open("seqkit.sh", "w") as f5:
        f5.writelines("#!/bin/sh\n")
        f5.writelines("source /PATH_TO_ENV\n")
        for read1, read2 in zip(r1_files, r2_files):
            f5.writelines(run_seqkit_pair(read1, read2) + "\n")     

In [15]:
subprocess.run(["sbatch", "-p", "short", "--ntasks=4", "--mem=40G", "seqkit.sh"])

Submitted batch job 3092692


CompletedProcess(args=['sbatch', '-p', 'short', '--ntasks=4', '--mem=40G', 'seqkit.sh'], returncode=0)

In [ ]:
def crispresso_JAK2_command(r1, r2):
    command = [
        "CRISPResso",
        "--fastq_r1", r1,
        "-r2", r2,
        "--min_average_read_quality", "30",
        "--plot_window_size", "11",
        "--cleavage_offset", "1",
        "--suppress_plots",
        "--amplicon_name", "JAK2_WT,JAK2_V617F",
        "--guide_seq", "TGTGGAGACGAGAGTAAGTA",
        "--amplicon_seq", 
        "TTATGGACAACAGTCAAACAACAATTCTTTGTACTTTTTTTTTTCCTTAGTCTTTCTTTGAAGCAGCAAGTATGATGAGCAAGCTTTCTCACAAGCATTTGGTTTTAAATTATGGAGTATGTGTCTGTGGAGACGAGAGTAAGTAAAACTACAGGCTTTCTAATGCCTTTCTCAGAGCATCTGTTTTT,"
        "TTATGGACAACAGTCAAACAACAATTCTTTGTACTTTTTTTTTTCCTTAGTCTTTCTTTGAAGCAGCAAGTATGATGAGCAAGCTTTCTCACAAGCATTTGGTTTTAAATTATGGAGTATGTTTCTGTGGAGACGAGAGTAAGTAAAACTACAGGCTTTCTAATGCCTTTCTCAGAGCATCTGTTTTT",
        "--quantification_window_center", "1",
        "--quantification_window_size", "15",
        "--guide_seq", "TGTGGAGACGAGAGTAAGTA",
        "--output_folder", "crispresso_outputs_JAK2"
    ]
    return command
    


In [ ]:
with open("crispresso.sh", "w") as f5:
        f5.writelines("#!/bin/sh\n")
        f5.writelines("source /PATH_TO_ENV\n")
        for read1, read2 in zip(r1_files, r2_files):
            f5.writelines(" ".join(crispresso_JAK2_command(read1, read2)) + "\n")

In [7]:
subprocess.run(["sbatch", "-p", "long", "--time=20:00:00", "--ntasks=8", "--mem=50G", "crispresso.sh"])

Submitted batch job 3106420


CompletedProcess(args=['sbatch', '-p', 'long', '--time=20:00:00', '--ntasks=8', '--mem=50G', 'crispresso.sh'], returncode=0)